In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
import plotting
import dataset_fctns
from scipy import stats
from scipy.stats import truncnorm

In [ ]:
def combine_columns(ds, Africa=False, numdays = 260):
    list_of_arrays = []
    if Africa:
        metadata_columns = ['year', 'Stations_id', 'lat', 'lon', 'WC SOS date']
        temp_variable = 'temperature'
    else:
        metadata_columns = ['year', 'Stations_id', 'lat', 'lon', 'SOS', 'WC SOS date']
        temp_variable = 't2m'
    for day_index in range(numdays):
        day_array = ds[metadata_columns + [f'NDVI interpolated at day {day_index}', f'{temp_variable} at day {day_index}', 'observed time to beginning of flowering']].set_index(['Stations_id', 'year']).to_xarray()
        day_array = day_array.rename({f'NDVI interpolated at day {day_index}': 'NDVI', f'{temp_variable} at day {day_index}': 't2m'})
        day_array = day_array.expand_dims('time')
        day_array = day_array.assign_coords(time=("time", [day_index]))
        list_of_arrays.append(day_array)
    return xr.concat(list_of_arrays, dim='time')
#ripeness_array#.expand_dims(dim={'time':200})
#ripeness

def MVI_array(da, index_variable = 'time', response_variable = 'NDVI', bins = np.arange(0, 201, 10), original_length = 1):
    binned = da.groupby_bins(index_variable, bins, labels = bins[:-1])
    maxs = binned.max()
    max_locs = binned.map(lambda arr: np.argmax(arr))
    max_locs[index_variable] = max_locs[f'{index_variable}_bins'] + max_locs[response_variable]*original_length
    #print(maxs[f'{index_variable}_bins'].shape, max_locs[index_variable].shape, maxs[response_variable].shape)
    #print(scipy.interpolate.interp1d())
    maxs['MVI NDVI'] = ((f'{index_variable}_bins'), np.interp(maxs[f'{index_variable}_bins'], max_locs[index_variable], maxs[response_variable]))
    return maxs

def to_temp_coords(da, new_coords = np.arange(0, 1701, 10)):
    da['time_index'] = da['time']#
    temps_maxed = da.groupby('t2m').max().set_coords('time_index')
    temps_interpolated = temps_maxed.interp(t2m=new_coords, kwargs={"fill_value": 0})
    return temps_interpolated

def append_frame(df, row):
    df.loc[-1] = row
    df.index = df.index + 1
    df = df.sort_index()
    return df

def convert_to_GDD_time(ds_inputs, numdays = 260, Africa = True, offset = 0, GDD_max = 2800, NDVI_max = 260):
    input_array = combine_columns(ds_inputs, numdays = numdays, Africa=Africa)
    GDD_bins = np.arange(0, GDD_max + 1, 100)
    output_df = pd.DataFrame(columns=['Stations_id', 'year', 'observed time to beginning of flowering GDD'] + [f'NDVI at GDD {GDD_bin}' for GDD_bin in GDD_bins[:-1]])
    count = 0
    for statid in input_array['Stations_id'].values:
        for year in input_array['year'].values:
            input_array_at_year = input_array.sel(Stations_id = statid, year = year)
            if np.isnan(input_array_at_year['t2m'].isel(time = 0)):
                continue
            count += 1
            if count % 20 == 0:
                print(f'#### {count} samples processed ####')
            maxs = MVI_array(input_array_at_year, index_variable = 'time', response_variable = 'NDVI', bins = np.arange(0, NDVI_max + 1, 10), original_length = 1)
            anthesis_day = maxs['observed time to beginning of flowering'].values[0].astype('timedelta64[D]') / np.timedelta64(1, 'D') + offset

            temps_interpolated = to_temp_coords(input_array_at_year, new_coords = np.arange(0, GDD_max + 1, 10))
            maxs = MVI_array(temps_interpolated, index_variable = 't2m', response_variable = 'NDVI', bins = GDD_bins, original_length = 10)
            anthesis_GDD = input_array_at_year.isel(time = int(anthesis_day))['t2m'].values

            output_df = append_frame(output_df, [statid, year, anthesis_GDD, *maxs['MVI NDVI'].values])
    return output_df

In [ ]:
GDD_LSP_list = []
for n in range(0, 1800, 100):
    GDD_LSP_list.append(pd.read_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\GDD_time_MODIS_savgol_NDVI_DE_{n}.csv'))

In [ ]:
pd.concat(GDD_LSP_list).to_csv('C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\for_baseline_tests\\GDD_time_MODIS_savgol_NDVI_DE.csv')

In [ ]:
import warnings
with warnings.catch_warnings(action="ignore"):
    GDD_time_SSA = convert_to_GDD_time(ds_inputs_SSA)#.loc[:10, :])
    GDD_time_SSA.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\GDD_time_MODIS_savgol_NDVI_SSA.csv')

In [ ]:
#import warnings
#with warnings.catch_warnings(action="ignore"):
#    for n in range(0, 1800, 100):
#        print(n)
#        GDD_time_DE = convert_to_GDD_time(ds_inputs_DE[n:(n + 100)], Africa=False, numdays=190)#.loc[:10, :])
#        GDD_time_DE.to_csv(f'C:\\Users\\wlwc1989\\Documents\\Phenology_Test_Notebooks\\phenology_dwd\\results_for_comparing\\GDD_time_MODIS_savgol_NDVI_DE_{n}.csv')

In [ ]:
input_array = combine_columns(ds_inputs_DE, numdays = 190)
fig, axs = plt.subplots(1, 2)
for statid in input_array['Stations_id'].values[[0, 10, 20, 30]]:
    for year in input_array['year'].values[0:7]:
        input_array_at_year = input_array.sel(Stations_id = statid, year = year)
        if np.isnan(input_array_at_year['t2m'].isel(time = 0)):
            continue
        time_binned = input_array_at_year.groupby_bins('time', np.arange(0, 191, 10), labels = np.arange(0, 181, 10))
        maxs = MVI_array(input_array_at_year, index_variable = 'time', response_variable = 'NDVI', bins = np.arange(0, 261, 10), original_length = 1)
        axs[0].plot(maxs['time_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_day = maxs['observed time to beginning of flowering'].values[0].astype('timedelta64[D]') / np.timedelta64(1, 'D')# + 60
        axs[0].axvline(anthesis_day)
        temps_interpolated = to_temp_coords(input_array_at_year, new_coords = np.arange(0, 3801, 10))
        maxs = MVI_array(temps_interpolated, index_variable = 't2m', response_variable = 'NDVI', bins = np.arange(0, 3801, 100), original_length = 10)
        axs[1].plot(maxs['t2m_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_GDD = input_array_at_year.isel(time = int(anthesis_day))['t2m'].values
        print(anthesis_GDD)
        axs[1].axvline(anthesis_GDD)

In [ ]:
input_array = combine_columns(ds_inputs_SSA, Africa=True)
fig, axs = plt.subplots(1, 2)
for statid in input_array['Stations_id'].values[0:15]:
    for year in input_array['year'].values[0:7]:
        input_array_at_year = input_array.sel(Stations_id = statid, year = year)
        if np.isnan(input_array_at_year['t2m'].isel(time = 0)):
            continue
        time_binned = input_array_at_year.groupby_bins('time', np.arange(0, 261, 10), labels = np.arange(0, 251, 10))
        maxs = MVI_array(input_array_at_year, index_variable = 'time', response_variable = 'NDVI', bins = np.arange(0, 261, 10), original_length = 1)
        axs[0].plot(maxs['time_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_day = maxs['observed time to beginning of flowering'].values[0].astype('timedelta64[D]') / np.timedelta64(1, 'D')# + 60
        axs[0].axvline(anthesis_day)
        temps_interpolated = to_temp_coords(input_array_at_year, new_coords = np.arange(0, 3801, 10))
        maxs = MVI_array(temps_interpolated, index_variable = 't2m', response_variable = 'NDVI', bins = np.arange(0, 3801, 100), original_length = 10)
        axs[1].plot(maxs['t2m_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_GDD = input_array_at_year.isel(time = int(anthesis_day))['t2m'].values
        print(anthesis_GDD)
        axs[1].axvline(anthesis_GDD)

In [ ]:
input_array = combine_columns(ds_inputs_SSA, Africa=True)
fig, axs = plt.subplots(1, 2)
for statid in input_array['Stations_id'].values[14:18]:
    for year in input_array['year'].values[0:8]:
        input_array_at_year = input_array.sel(Stations_id = statid, year = year)
        if np.isnan(input_array_at_year['t2m'].isel(time = 0)):
            continue
        #time_binned = input_array_at_year.groupby_bins('time', np.arange(0, 261, 10), labels = np.arange(0, 251, 10))
        maxs = MVI_array(input_array_at_year, index_variable = 'time', response_variable = 'NDVI', bins = np.arange(0, 261, 10), original_length = 1)
        axs[0].plot(maxs['time_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_day = maxs['observed time to beginning of flowering'].values[0].astype('timedelta64[D]') / np.timedelta64(1, 'D') + 60
        axs[0].axvline(anthesis_day)
        temps_interpolated = to_temp_coords(input_array_at_year, new_coords = np.arange(0, 5801, 10))
        maxs = MVI_array(temps_interpolated, index_variable = 't2m', response_variable = 'NDVI', bins = np.arange(0, 5801, 100), original_length = 10)
        axs[1].plot(maxs['t2m_bins'], maxs['MVI NDVI'], color = 'red', alpha = 0.5)
        anthesis_GDD = input_array_at_year.isel(time = int(anthesis_day))['t2m'].values
        print(anthesis_GDD)
        axs[1].axvline(anthesis_GDD)